## Import packages and other dataframes

In [ ]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import pubchempy as pcp
import unicodedata
from bs4 import BeautifulSoup


# searching wikidata for compounds from a specific speceies

Search is based on wikidata search using the capsicinoid code wd:Q20918454. This code can be changed for different molecular classes

In [ ]:
def search_species_taxon():
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)
    query = """#title: Which organisms contain capsaicinoid scaffolds?
SELECT ?structure ?structureLabel ?structure_inchikey ?structure_smiles_canonical ?structure_smiles_isomeric ?taxon ?taxon_name ?reference ?reference_doi ?chemical_formula WHERE {
  VALUES ?SUBSTRUCTURE {
    wd:Q20918454
  }
  ?structure wdt:P279 ?SUBSTRUCTURE.
  hint:Prior hint:runFirst "true"^^xsd:boolean.
  ?structure p:P703 ?statement;
    wdt:P233 ?structure_smiles_canonical;
    wdt:P235 ?structure_inchikey;
    wdt:P274 ?chemical_formula.
  OPTIONAL { ?structure wdt:P2017 ?structure_smiles_isomeric. }
  ?statement ps:P703 ?taxon.
  ?taxon wdt:P225 ?taxon_name.
  ?statement (prov:wasDerivedFrom/pr:P248) ?reference.
  ?reference wdt:P356 ?reference_doi.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""


    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"] #results are stored as results and bindings 

#results are a list of dictionaries, where we need to sepaarate 
results = search_species_taxon()

cleaned_results = []
for row in results:
    cleaned_dict = {}
    for name, item in row.items():
        cleaned_dict[name] = item["value"]
        # print(name)
        # print(type(item["value"]))
    cleaned_results.append(cleaned_dict)


wiki_df = pd.DataFrame.from_dict(cleaned_results)



In [ ]:
wiki_df = pd.DataFrame(cleaned_results)
# wiki_df

## Reducing duplicate entries based on split InChiKey

Only the first data value is kept, this will remove information from the reference DOI. If more information is needed go back orignial dataframe.

In [ ]:
# Split the InChIKeys and create a new column
wiki_df['InChIKey_Split'] = wiki_df['structure_inchikey'].str.split('-').str[0]


# Drop duplicate rows based on the 'SMILES' column
wiki_df_uni= wiki_df.drop_duplicates(subset='InChIKey_Split', keep='first', ignore_index=True)


# Print the unique DataFrame
# wiki_df_uni


In [ ]:
#notmalize all string to account utf-8 molecular formulas 
def normalize_string(string):
    normalized_string = ""
    for char in string:
        normalized_char = unicodedata.normalize('NFKD', char).encode('ascii', 'ignore').decode('utf-8')
        if normalized_char.isalnum():
            normalized_string += normalized_char
    return normalized_string

# Assuming 'Molecular Formula' is a column in the 'wiki_df_short' DataFrame
wiki_df_uni['chemical_formula'] = wiki_df_uni['chemical_formula'].apply(normalize_string)

wiki_df_uni.rename(columns={'chemical_formula': 'Molecular Formula','structure_smiles_canonical':'Database_SMILES'}, inplace=True)

In [ ]:
wiki_df_uni.to_csv('Data/Dereplication/Capsaicinoid_dereplication/Output/wiki_df_uni.csv', index=False)

# Import tables form reaxys

Note: currently waiting to hear about getting rexays API access 

When exporting, select Excel and hit data, I used range for the exact number f moleucles found and include structure. If you exclude structure you do not obtain the SMILES in the export

In [ ]:
rex_df = pd.read_excel('Data/Dereplication/Capsaicinoid_dereplication/Input/Reaxys_Cap.xlsx', usecols=['SMILES', 'Chemical Name','Molecular Formula','InChI Key','Isolated from Natural Source','References' ])
# rex_df

In [ ]:
#split inchi keys, keeping only the first half
rex_df['InChIKey_Split'] = rex_df['InChI Key'].str.split('-').str[0]

rex_df.rename(columns={'SMILES': 'Database_SMILES'}, inplace=True)
rex_df

In [ ]:

# Drop duplicate rows based on the 'SMILES' column
rex_df_uni = rex_df.drop_duplicates(subset='InChIKey_Split', keep='first', ignore_index=True)
rex_df_uni.dropna(inplace=True)
rex_df_uni

In [ ]:
rex_df_uni.to_csv('Data/Dereplication/Capsaicinoid_dereplication/Output/rexays_df_uni.csv', index=False)

# Merging and cleaning database table

In [ ]:
rex_df_uni = pd.read_csv('Data/Dereplication/Capsaicinoid_dereplication/Output/rexays_df_uni.csv', usecols={'Database_SMILES', 'Chemical Name','Molecular Formula','InChI Key','InChIKey_Split','Isolated from Natural Source','References'})

wiki_df_uni = pd.read_csv('Data/Dereplication/Capsaicinoid_dereplication/Output/wiki_df_uni.csv')

In [ ]:
# Merge the data frames on 'InChIKey_Split'
database_df = pd.merge(rex_df_uni, wiki_df_uni, on=[ 'Molecular Formula', 'InChIKey_Split'], how='outer')
database_df

In [ ]:
database_df['Database_SMILES'] = database_df['Database_SMILES_x'].combine_first(database_df['Database_SMILES_y'])
database_df['InChI Key']= database_df['InChI Key'].combine_first(database_df['structure_inchikey'])
database_df['Chemical Name']= database_df['Chemical Name'].combine_first(database_df['structureLabel'])
database_df['Isolated from Natural Source']= database_df['Isolated from Natural Source'].combine_first(database_df['taxon_name'])
database_df['References']= database_df['References'].combine_first(database_df['reference_doi'])

In [ ]:
database_df.to_csv('Data/Dereplication/Capsaicinoid_dereplication/Output/database_df.csv', columns={'Database_SMILES', 'Chemical Name','Molecular Formula','InChI Key','Isolated from Natural Source','References'})

After export, ensure that all entries make sense. It is possible that some compounds obtained from Wikidata do not make sense for your current analysis.

## Group columns on Molecular Formula due to isomers

In [ ]:
#read final database file after manual cleaning
database_df = pd.read_csv('Data/Dereplication/Capsaicinoid_dereplication/Output/database_df.csv')

In [ ]:
#convert dataframe to string for grouping
database_df = database_df.astype(str)


concat_database_df = database_df.groupby('Molecular Formula',as_index=False).agg(lambda x: ';'.join(x.unique()))


# Assuming 'column_name' is the name of the column you want to split
concat_database_df['Database_SMILES_First_Entry'] = concat_database_df['Database_SMILES'].str.split(';').str[0]

concat_database_df.drop(columns=['Unnamed: 0'], inplace=True)
# concat_database_df

In [ ]:
concat_database_df.to_csv('Data/Dereplication/Capsaicinoid_dereplication/Output/Capsaicinoids_database_final.csv', index=False, encoding='utf-8-sig',)